In [13]:
from mimetypes import init
from pyexpat import model
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss

import matplotlib.pyplot as plt
import random
import pdb
import argparse
from torch import optim

class MNISTDatasetWithConcepts(Dataset):
	def __init__(self,split,num_classes,transform):
		isTrain = False
		if split == "train":
			isTrain=True
		self.device = "cuda" if torch.cuda.is_available() else "cpu"
		self.data = MNIST(root = "./synthetic_datasets",train=isTrain, download=True)
		self.num_classes = num_classes
		self.transform = transform
		# print(len(set([self.data[i][1] for i in range(len(self.data))])))

	def __len__(self):
		return len(self.data)

	def __getitem__(self,idx):
		img,label = self.data[idx][0],self.data[idx][1]
		onehot = torch.zeros((self.num_classes,))
		onehot[label] =1
		concept = self.make_concepts_mnist(label)
		label = onehot.to(self.device)
		return [self.transform(img).to(self.device),label,concept]

	def make_concepts_mnist(self,label):
		if label == 0:
			hard_label = torch.tensor([1,0,0,0,0,0,0,0,0,0]+[1,0,0,1,0,0,0,0])
		elif label == 1:
			hard_label = torch.tensor([0,1,0,0,0,0,0,0,0,0]+[0,1,0,0,1,1,0,1])
		elif label == 2:
			hard_label = torch.tensor([0,0,1,0,0,0,0,0,0,0]+[1,1,0,0,1,0,0,1])
		elif label == 3:
			hard_label = torch.tensor([0,0,0,1,0,0,0,0,0,0]+[1,0,0,0,0,0,0,0])
		elif label == 4:
			hard_label = torch.tensor([0,0,0,0,1,0,0,0,0,0]+[0,1,1,0,1,1,1,0])
		elif label == 5:
			hard_label = torch.tensor([0,0,0,0,0,1,0,0,0,0]+[1,1,0,0,1,1,1,0])
		elif label == 6:
			hard_label = torch.tensor([0,0,0,0,0,0,1,0,0,0]+[1,0,1,1,0,0,0,1])
		elif label == 7:
			hard_label = torch.tensor([0,0,0,0,0,0,0,1,0,0]+[0,1,0,0,1,0,1,0])
		elif label == 8:
			hard_label = torch.tensor([0,0,0,0,0,0,0,0,1,0]+[1,0,1,1,0,0,1,1])
		elif label == 9:
			hard_label = torch.tensor([0,0,0,0,0,0,0,0,0,1]+[1,0,1,1,0,0,1,0])

		# pdb.set_trace()

		# hard_label = torch.zeros((self.num_classes,))
		# hard_label[label] = 1
		hard_label = hard_label.float().to(self.device)
		return hard_label


def load_mnist_dataloader(split,bsz):
	dataset = MNISTDatasetWithConcepts(split = split, num_classes = 10, transform=ToTensor())
	dataloader = DataLoader(dataset, batch_size=bsz, shuffle=True)
	return dataloader


In [2]:
train_loader = load_mnist_dataloader(split = "train",bsz=64)
test_loader = load_mnist_dataloader(split = "test",bsz=64)

100%|██████████| 9912422/9912422 [00:00<00:00, 149444329.10it/s]

Extracting ./synthetic_datasets/MNIST/raw/train-images-idx3-ubyte.gz to ./synthetic_datasets/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 38419186.12it/s]


Extracting ./synthetic_datasets/MNIST/raw/train-labels-idx1-ubyte.gz to ./synthetic_datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 47889673.34it/s]

Extracting ./synthetic_datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to ./synthetic_datasets/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 7153784.74it/s]


Extracting ./synthetic_datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./synthetic_datasets/MNIST/raw



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class g(nn.Module):
    """The network g consists of 2 convolutional
    layers with 32 channels each, along with a maxpool
    layer in between followed by a fully connected
    layer."""

    def __init__(self, n_concepts):
        super(g, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.fc1 = nn.Linear(800, 128)
        self.fc2 = nn.Linear(128, 10+n_concepts*2)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        non_overlapping, overlapping = x[:,:10], x[:,10:]
        return non_overlapping, overlapping

class f(nn.Module):
    def __init__(self, input_size) -> None:
        super(f, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

def split_concepts(concepts):
    return concepts[:,:10], concepts[:,10:]

class Sequential(nn.Module):
    def __init__(self, n_concepts):
        super(Sequential, self).__init__()
        self.g_model = g(n_concepts).to(device)
        self.f_model = f(10+n_concepts*2).to(device)

        # Defining the training parameters for the concepts model g
        self.g_optimizer = torch.optim.Adam(self.g_model.parameters(), lr=1e-4)
        self.g_criterion = nn.CrossEntropyLoss()

        # Defining the training parameters for the prediction model f
        self.learned_g = False
        self.f_optimizer = torch.optim.Adam(self.f_model.parameters(), lr=1e-4)
        self.f_criterion = nn.CrossEntropyLoss()
        self.name = 'sequential'

    def split_concepts(self, concepts):
        return concepts[:,:10], concepts[:,10:]

    def loss_overlapping(self, overlapping, y_true):
        total_loss = 0
        for i in range(0, overlapping.shape[1], 2):
            loss = self.g_criterion(overlapping[:, i:i+2], y_true[:, int(i/2)].long())
            total_loss += loss

        return total_loss

    def train_g(self, train_loader, epochs):
        self.g_model.train()
        for epoch in range(epochs):
            for images, labels, concepts in train_loader:
                images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
                y_true_non_overlapping, y_true_overlapping = self.split_concepts(concepts)

                self.g_optimizer.zero_grad()
                non_overlapping, overlapping = self.g_model(images)

                # Calculate the loss for non_overlaaping concepts
                loss_non_overlapping = self.g_criterion(non_overlapping, y_true_non_overlapping.argmax(dim=1))

                # Calculate the loss for overlapping concepts
                loss_overlapping = self.loss_overlapping(overlapping, y_true_overlapping)

                # Calculate the total loss
                loss = loss_non_overlapping + loss_overlapping
                loss.backward()
                self.g_optimizer.step()
            print(f"Epoch: {epoch+1}, Loss: {loss.item():.3f}")
        self.learned_g = True

    def calc_acc_non_overlapping(self, test_loader):
        self.g_model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, _, concepts in test_loader:
                images, concepts = images.to(device), concepts.to(device)
                y_true_non_overlapping, _ = self.split_concepts(concepts)
                non_overlapping, _ = self.g_model(images)
                y_pred_non_overlapping = non_overlapping.argmax(dim=1)
                correct += (y_pred_non_overlapping == y_true_non_overlapping.argmax(dim=1)).sum().item()
                total += y_true_non_overlapping.size(0)

        return correct / total if total > 0 else 0

    def calc_acc_overlapping(self, test_loader):
        self.g_model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, _, concepts in test_loader:
                images, concepts = images.to(device), concepts.to(device)
                _, y_true_overlapping = self.split_concepts(concepts)
                _, overlapping = self.g_model(images)

                for i in range(0, overlapping.shape[1], 2):
                    y_pred_overlapping = overlapping[:, i:i+2].argmax(dim=1)
                    correct += (y_pred_overlapping == y_true_overlapping[:, int(i/2)]).sum().item()
                    total += y_true_overlapping.size(0)

        return correct / total if total > 0 else 0

    def calc_acc_g(self, test_loader):
        self.g_model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for images, _, concepts in test_loader:
                images, concepts = images.to(device), concepts.to(device)
                y_true_non_overlapping, y_true_overlapping = self.split_concepts(concepts)
                non_overlapping, overlapping = self.g_model(images)

                y_pred_non_overlapping = non_overlapping.argmax(dim=1)
                correct += (y_pred_non_overlapping == y_true_non_overlapping.argmax(dim=1)).sum().item()
                total += y_true_non_overlapping.size(0)

                for i in range(0, overlapping.shape[1], 2):
                    y_pred_overlapping = overlapping[:, i:i+2].argmax(dim=1)
                    correct += (y_pred_overlapping == y_true_overlapping[:, int(i/2)]).sum().item()
                    total += y_true_overlapping.size(0)

        return correct / total if total > 0 else 0

    def train_f(self, train_loader, epochs):
        if not self.learned_g:
            raise "You have to train g before training f in sequential training"
        self.f_model.train()
        for epoch in range(epochs):
            for images, labels, _ in train_loader:
                images, labels = images.to(device), labels.to(device)
                self.f_optimizer.zero_grad()

                non_overlapping, overlapping = self.g_model(images)
                # Concatenate the concepts
                overlapping_concat = torch.cat([non_overlapping, overlapping], dim=1)
                y_pred = self.f_model(overlapping_concat)
                loss = self.f_criterion(y_pred, labels.argmax(dim=1))
                loss.backward()
                self.f_optimizer.step()
            print(f"Epoch: {epoch+1}, Loss: {loss.item():.3f}")

    def calc_acc_prediction(self, test_loader, delta=None):
        accuracies = []
        for images, labels, _ in test_loader:
            images, labels = images.to(device), labels.to(device)
            if delta is not None:
                y_pred = self.forward(images + delta[:images.shape[0]]).argmax(dim=1)
            else:
                y_pred = self.forward(images).argmax(dim=1)
            acc = (y_pred == labels.argmax(dim=1)).float().mean().item()
            accuracies.append(acc)
        return sum(accuracies)/len(accuracies)

    def train(self, train_loader, epochs=20):
        self.train_g(train_loader, epochs)
        self.train_f(train_loader, epochs)
    def forward(self, x):
        non_overlapping, overlapping = self.g_model(x)
        # Concatenate the concepts
        overlapping = torch.cat([non_overlapping, overlapping], dim=1)
        return self.f_model(overlapping)

    def save_g(self):
        torch.save(self.g_model.state_dict(), "g_model1.pth")

    def load_g(self, path):
        self.g_model.load_state_dict(torch.load(path))
        self.learned_g = True

    def save_f(self):
        torch.save(self.f_model.state_dict(), "f_model1.pth")

    def load_f(self, path):
        self.f_model.load_state_dict(torch.load(path))

## testing the accuracy before adversarial training

In [ ]:

n_concepts = 8
model = Sequential(8)
model.load_state_dict(torch.load("sequential3.pth"))

FileNotFoundError: ignored

In [ ]:
def pgd_linf_targ(model, data_loader, epsilon, alpha, num_iter, y_targ, example):
    """
      Training on the whole test set
    """
    delta = torch.zeros_like(example, requires_grad=True)
    for t in range(num_iter):
      print(f"Running iteration {t}")
      for X, y, concepts in data_loader:
        X, y, concepts = images.to(device), labels.to(device), concepts.to(device)
        yp = model(X + delta)
        loss = 2*yp[:, y_targ].sum() - yp.sum()
        loss.backward()

        delta = delta + alpha * delta.grad.detach().sign()
        delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

        # Clear gradients after updating delta
        if delta.grad is not None:
            delta.grad.zero_()
    return delta.detach()


for images, labels, concepts in test_loader:
    images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
    break
delta = pgd_linf_targ(model, test_loader, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2, example=images)
yp = model(images + delta)

Running iteration 0
Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
Running iteration 10
Running iteration 11
Running iteration 12
Running iteration 13
Running iteration 14
Running iteration 15
Running iteration 16
Running iteration 17
Running iteration 18
Running iteration 19


In [ ]:
model.calc_acc_prediction(test_loader)

0.9892515923566879

In [ ]:
model.calc_acc_prediction(test_loader, delta)

0.2428343949044586

In [ ]:
def epoch_adversarial(loader, model, attack, opt=None, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y,_ in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y)
        yp = model(X+delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        total_err += (yp.max(dim=1)[1] != y.argmax(dim=1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


In [ ]:
def pgd_linf_targ(model, X,y, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2):
    """
      Training on the whole test set
    """
    X, y = X.to(device), y.to(device)
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):

      yp = model(X + delta)
      loss = 2*yp[:, y_targ].sum() - yp.sum()
      loss.backward()

      delta = delta + alpha * delta.grad.detach().sign()
      delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

      # Clear gradients after updating delta
      if delta.grad is not None:
          delta.grad.zero_()
    return delta.detach()


In [ ]:
opt = optim.Adam(model.parameters(), lr=1e-4)
for t in range(10):
    print(f"Running iteration {t}")
    train_err, train_loss = epoch_adversarial(train_loader, model, pgd_linf_targ, opt)
    adv_err, adv_loss = epoch_adversarial(test_loader, model, pgd_linf_targ)
    if t == 4:
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-5
    print(*("{:.6f}".format(i) for i in (train_err, adv_err)), sep="\t")
torch.save(model.state_dict(), "model_robust.pt")

Running iteration 0
0.018733	0.019000
Running iteration 1
0.016917	0.018000
Running iteration 2
0.014700	0.014900
Running iteration 3
0.013517	0.015000
Running iteration 4
0.012850	0.016500
Running iteration 5
0.010500	0.014400
Running iteration 6
0.010117	0.014000
Running iteration 7
0.009667	0.014700
Running iteration 8
0.009033	0.014500
Running iteration 9
0.009550	0.014000


In [ ]:
model.calc_acc_prediction(test_loader)

0.9299363057324841

In [ ]:
model.calc_acc_prediction(test_loader, delta)

0.4476512738853503

# Adversarially training the joint model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Joint(nn.Module):
    def __init__(self, n_concepts):
        super(Joint, self).__init__()
        self.g_model = g(n_concepts).to(device)
        self.f_model = f(10+n_concepts*2).to(device)

        self.name = 'joint'

    def forward(self, x):
        non_overlapping, overlapping = self.g_model(x)
        overlapping_concat = torch.cat([non_overlapping, overlapping], dim=1)
        y_pred = self.f_model(overlapping_concat)
        return y_pred, non_overlapping, overlapping


    def loss_overlapping(self, overlapping, y_true):
        total_loss = 0
        concept_idx = 0
        # print(overlapping.shape)
        for i in range(0, overlapping.shape[1], 2):
            loss = nn.CrossEntropyLoss()(overlapping[:, i:i+2], y_true[:, concept_idx].long())
            concept_idx += 1
            total_loss += loss

        return total_loss




### Testing the accuracy of the model before the adversarial training

In [ ]:
joint_model = Joint(8)
joint_model.load_state_dict(torch.load(r"/content/joint_model2.pth"))

<All keys matched successfully>

In [ ]:
def test_joint(model, test_loader):
    model.eval()
    accuracies = []
    for images, labels, concepts in test_loader:
        images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
        y_pred, _, _ = model(images)
        acc = (y_pred.argmax(dim=1) == labels.argmax(dim=1)).float().mean().item()
        accuracies.append(acc)
    return sum(accuracies)/len(accuracies)
test_joint(joint_model, test_loader)

0.9892515923566879

In [ ]:
def pgd_linf_targ(model, data_loader, epsilon, alpha, num_iter, y_targ, example):
    """
      Training on the whole test set
    """
    delta = torch.zeros_like(example, requires_grad=True)
    for t in range(num_iter):
      print(f"Running iteration {t}")
      for X, y, concepts in data_loader:
        X, y, concepts = images.to(device), labels.to(device), concepts.to(device)
        yp, _, _ = model(X + delta)
        loss = 2*yp[:, y_targ].sum() - yp.sum()
        loss.backward()

        delta = delta + alpha * delta.grad.detach().sign()
        delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

        # Clear gradients after updating delta
        if delta.grad is not None:
            delta.grad.zero_()
    return delta.detach()


for images, labels, concepts in test_loader:
    images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
    break
delta = pgd_linf_targ(joint_model, test_loader, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2, example=images)


Running iteration 0
Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
Running iteration 10
Running iteration 11
Running iteration 12
Running iteration 13
Running iteration 14
Running iteration 15
Running iteration 16
Running iteration 17
Running iteration 18
Running iteration 19


In [ ]:
def test_joint_addv(model, test_loader, delta):
    model.eval()
    accuracies = []
    for images, labels, concepts in test_loader:
        images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
        y_pred, _, _ = model(images + delta[:images.shape[0]])
        acc = (y_pred.argmax(dim=1) == labels.argmax(dim=1)).float().mean().item()
        accuracies.append(acc)
    return sum(accuracies)/len(accuracies)
test_joint_addv(joint_model, test_loader, delta)

0.16660031847133758

### Adversarially training the joint model

In [ ]:
def pgd_linf_targ(model, X,y, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2):
    """
      Training on the whole test set
    """
    X, y = X.to(device), y.to(device)
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):
      yp, _, _ = model(X + delta)
      loss = 2*yp[:, y_targ].sum() - yp.sum()
      loss.backward()

      delta = delta + alpha * delta.grad.detach().sign()
      delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

      # Clear gradients after updating delta
      if delta.grad is not None:
          delta.grad.zero_()
    return delta.detach()


In [ ]:
def epoch_adversarial(loader, model, attack, opt=None, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y,_ in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y)
        yp, _, _ = model(X+delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        total_err += (yp.max(dim=1)[1] != y.argmax(dim=1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


In [ ]:
opt = optim.Adam(joint_model.parameters(), lr=1e-4)
for t in range(10):
    print(f"Running iteration {t}")
    train_err, train_loss = epoch_adversarial(train_loader, joint_model, pgd_linf_targ, opt)
    adv_err, adv_loss = epoch_adversarial(test_loader, joint_model, pgd_linf_targ)
    if t == 4:
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-5
    print(*("{:.6f}".format(i) for i in (train_err, adv_err)), sep="\t")
torch.save(joint_model.state_dict(), "joint_robust.pt")

Running iteration 0
0.018900	0.018700
Running iteration 1
0.014833	0.016600
Running iteration 2
0.013683	0.015400
Running iteration 3
0.012183	0.014300
Running iteration 4
0.011600	0.015600
Running iteration 5
0.008800	0.013400
Running iteration 6
0.008200	0.013000
Running iteration 7
0.007950	0.012500
Running iteration 8
0.007567	0.012700
Running iteration 9
0.007733	0.012300


In [ ]:
test_joint_addv(joint_model, test_loader, delta)

0.5200039808917197

In [ ]:
test_joint(joint_model, test_loader)

0.856687898089172

# Adversarially training CNN model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
  def __init__(self):
      super(CNN, self).__init__()
      self.conv1 = nn.Conv2d(1, 32, 3, 1)
      self.conv2 = nn.Conv2d(32, 32, 3, 1)
      self.fc1 = nn.Linear(800, 128)
      self.fc2 = nn.Linear(128, 10)

      self.name = 'CNN'

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.fc2(x)
    return x



In [5]:
cnn_model = CNN().to(device)
cnn_model.load_state_dict(torch.load('cnn1.pth'))

<All keys matched successfully>

In [6]:
def test_cnn(model, test_loader):
  model.eval()
  accuracies = []
  for images, labels, _ in test_loader:
    images, labels = images.to(device), labels.to(device)
    y_pred = model(images).argmax(dim=1)
    acc = (y_pred == labels.argmax(dim=1)).float().mean().item()
    accuracies.append(acc)
  return sum(accuracies)/len(accuracies)
test_cnn(cnn_model, test_loader)

0.9886544585987261

In [7]:
def pgd_linf_targ(model, data_loader, epsilon, alpha, num_iter, y_targ, example):
    delta = torch.zeros_like(example, requires_grad=True)
    for t in range(num_iter):
      print(f"Running iteration {t}")
      for X, y, concepts in data_loader:
        X, y, concepts = images.to(device), labels.to(device), concepts.to(device)
        yp = model(X + delta)
        loss = 2*yp[:, y_targ].sum() - yp.sum()
        loss.backward()

        delta = delta + alpha * delta.grad.detach().sign()
        delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

        # Clear gradients after updating delta
        if delta.grad is not None:
            delta.grad.zero_()
    return delta.detach()


for images, labels, concepts in test_loader:
    images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
    break
delta = pgd_linf_targ(cnn_model, test_loader, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2, example=images)

Running iteration 0
Running iteration 1
Running iteration 2
Running iteration 3
Running iteration 4
Running iteration 5
Running iteration 6
Running iteration 7
Running iteration 8
Running iteration 9
Running iteration 10
Running iteration 11
Running iteration 12
Running iteration 13
Running iteration 14
Running iteration 15
Running iteration 16
Running iteration 17
Running iteration 18
Running iteration 19


In [8]:
def test_cnn_addv(model, test_loader, delta):
    model.eval()
    accuracies = []
    for images, labels, concepts in test_loader:
        images, labels, concepts = images.to(device), labels.to(device), concepts.to(device)
        y_pred = model(images + delta[:images.shape[0]])
        acc = (y_pred.argmax(dim=1) == labels.argmax(dim=1)).float().mean().item()
        accuracies.append(acc)
    return sum(accuracies)/len(accuracies)
test_cnn_addv(cnn_model, test_loader, delta)

0.11624203821656051

In [9]:
def epoch_adversarial(loader, model, attack, opt=None, **kwargs):
    """Adversarial training/evaluation epoch over the dataset"""
    total_loss, total_err = 0.,0.
    for X,y,_ in loader:
        X,y = X.to(device), y.to(device)
        delta = attack(model, X, y)
        yp = model(X+delta)
        loss = nn.CrossEntropyLoss()(yp,y)
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        total_err += (yp.max(dim=1)[1] != y.argmax(dim=1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


In [10]:
def pgd_linf_targ(model, X,y, epsilon=0.4, alpha=1e-2, num_iter=20, y_targ=2):
    """
      Training on the whole test set
    """
    X, y = X.to(device), y.to(device)
    delta = torch.zeros_like(X, requires_grad=True)
    for t in range(num_iter):

      yp = model(X + delta)
      loss = 2*yp[:, y_targ].sum() - yp.sum()
      loss.backward()

      delta = delta + alpha * delta.grad.detach().sign()
      delta = delta.clamp(-epsilon, epsilon).detach().requires_grad_(True)

      # Clear gradients after updating delta
      if delta.grad is not None:
          delta.grad.zero_()
    return delta.detach()


In [14]:
opt = optim.Adam(cnn_model.parameters(), lr=1e-4)
for t in range(10):
    print(f"Running iteration {t}")
    train_err, train_loss = epoch_adversarial(train_loader, cnn_model, pgd_linf_targ, opt)
    adv_err, adv_loss = epoch_adversarial(test_loader, cnn_model, pgd_linf_targ)
    if t == 4:
        for param_group in opt.param_groups:
            param_group["lr"] = 1e-5
    print(*("{:.6f}".format(i) for i in (train_err, adv_err)), sep="\t")
torch.save(cnn_model.state_dict(), "cnn_robust.pt")

Running iteration 0
0.022183	0.017000
Running iteration 1
0.012850	0.017700
Running iteration 2
0.011750	0.015200
Running iteration 3
0.010233	0.015700
Running iteration 4
0.009867	0.013200
Running iteration 5
0.007383	0.013200
Running iteration 6
0.007283	0.013200
Running iteration 7
0.006950	0.013400
Running iteration 8
0.006883	0.012800
Running iteration 9
0.006783	0.013300


In [16]:
test_cnn(cnn_model, test_loader)

0.8883359872611465

In [15]:
test_cnn_addv(cnn_model, test_loader, delta)

0.6283837579617835